# Analyze Fundamental Stock Data

In [336]:
import pandas as pd
from eod import EodHistoricalData
from functools import reduce
import plotly.express as px
from datetime import datetime, timedelta
import requests

# Importing and assigning the api key
with open("../eodHistoricalData-API.txt", "r") as f:
    api_key = f.read()
    
# EOD Historical Data client
client = EodHistoricalData(api_key)

# Formatting Fundamental Data

In [324]:
def getFundamentals(ticker):
    """
    Returns the fundamental data from the financial data API.  Combines the quarterly balance 
    sheet, cash flow, income statement, and earnings for a specific stock ticker.
    """
    
    # Getting data
    fund_data = client.get_fundamental_equity(ticker)
    
    # Financials
    bal = pd.DataFrame(fund_data['Financials']['Balance_Sheet']['quarterly']).T
    
    cf = pd.DataFrame(fund_data['Financials']['Cash_Flow']['quarterly']).T
    
    inc = pd.DataFrame(fund_data['Financials']['Income_Statement']['quarterly']).T
    
    # Earnings
    earn = pd.DataFrame(fund_data['Earnings']['History']).T
    
    # Merging them together
    df = reduce(
        lambda left,right: pd.merge(
            left,
            right,
            left_index=True, 
            right_index=True, 
            how='outer',
            suffixes=('', '_drop')
        ), 
        [bal, cf, inc, earn]
    )
    
    # Dropping redundant date and duplicate columns
    dup_cols = [i for i in df.columns if "date" in i or "Date" in i or "_drop" in i]
    
    df = df.drop(dup_cols, axis=1)
    
    return df

In [409]:
def getPrices(df, ticker):
    """
    Gets the stock price at the time for each date in the financial statements for
    the given ticker and dataframe of financial information.
    """
    # Getting stock price at the time
    prices = client.get_prices_eod(ticker, period='d')
    
    
    prices = pd.DataFrame(prices).set_index('date')[['adjusted_close', 'close', 'volume']]

    # Converting to date time
    prices.index = pd.to_datetime(prices.index)

    # Filling in missing price data
    prices = prices.reindex(
        pd.date_range(prices.index[0], prices.index[-1]),
        method='ffill'
    )
    
    # Converting back to string for merging later
    prices.index = prices.index.strftime("%Y-%m-%d")
        
    price_dates = [i for i in prices.index if i in df.index]
    
    prices = prices.loc[price_dates]

    # Joining together
    df = df.join(prices, how='outer')
    
    return df

In [406]:
def formatFundamentals(ticker, dropna=False):
    """
    Formats the given ticker's fundamental and price data. Cleans up the data by dropping
    any empty/nan values if requested.
    """
    
    # Getting fundamental data
    fund_data = getFundamentals(ticker)
    
    # Getting accompanying price data
    df = getPriceData(ticker, fund_data)
    
    # Dropping if all items are na in respective row
    df = df.dropna(how='all')
    
    if dropna:
        # Dropping mostly nan columns and rows if requested
        df = df.dropna(
            axis=0,
            thresh=round(df.shape[0]*.3) # If 50% of the values in the row are Nans, drop the whole row
        ).dropna(
            axis=1,
            thresh=round(df.shape[1]*.3) # If 50% of the values in the columns are Nans, drop the whole column
        )
    
    return df

In [407]:
df = formatFundamentals("AAPL", dropna=True)

In [408]:
df

,currency_symbol,totalAssets,intangibleAssets,otherCurrentAssets,totalLiab,totalStockholderEquity,deferredLongTermLiab,otherCurrentLiab,commonStock,retainedEarnings,...,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares,currency,epsActual,epsEstimate,epsDifference,surprisePercent,adjusted_close,close,volume
1989-12-31,None,2813300000.0,None,None,1289500000.0,1523800000.0,None,None,None,1247500000.0,...,None,None,NaN,NaN,NaN,NaN,NaN,0.2520,35.2520,152409593.0
1990-03-31,None,2775700000.0,None,None,1224800000.0,1550900000.0,None,None,None,1341300000.0,...,None,None,NaN,NaN,NaN,NaN,NaN,0.2887,40.2528,223350390.0
1990-06-30,None,2889500000.0,None,None,1382700000.0,1506800000.0,None,None,None,1334800000.0,...,None,None,NaN,NaN,NaN,NaN,NaN,0.3219,44.7496,325191986.0
1990-09-30,None,2975700000.0,None,None,1528900000.0,1446800000.0,None,None,None,1312200000.0,...,None,None,NaN,NaN,NaN,NaN,NaN,0.2093,29.0024,176041592.0
1990-12-31,None,3157300000.0,None,None,1612200000.0,1545100000.0,None,None,None,1405300000.0,...,None,None,NaN,NaN,NaN,NaN,NaN,0.3114,43.0024,44273598.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-30,USD,317344000000.0,None,10987000000.0,245062000000.0,72282000000.0,None,41318000000.0,48696000000.0,24136000000.0,...,11253000000.0,11253000000.0,USD,0.65,0.5100,0.1400,27.4510,90.4541,364.8000,140223280.0
2020-09-30,USD,323888000000.0,None,11264000000.0,258549000000.0,65339000000.0,None,42684000000.0,50779000000.0,14966000000.0,...,12673000000.0,12673000000.0,USD,0.73,0.6987,0.0313,4.4797,115.0699,115.8100,142675203.0
2020-12-31,USD,354054000000.0,None,13687000000.0,287830000000.0,66224000000.0,None,48504000000.0,51744000000.0,14301000000.0,...,28755000000.0,28755000000.0,USD,1.68,1.4158,0.2642,18.6608,132.0695,132.6900,99116594.0
2021-03-31,USD,337158000000.0,None,13376000000.0,267980000000.0,69178000000.0,None,45660000000.0,54203000000.0,15261000000.0,...,23630000000.0,23630000000.0,USD,1.40,0.9888,0.4112,41.5858,121.7604,122.1500,118323797.0


# Getting Fundamentals from Multiple Companies

In [356]:
def getMultipleFunds(tickers, api_token):
    """
    Gets fundamental data from multiple stock tickers given as a list. Returns
    a large dataframe containing the concatenated information for all the given
    tickers.
    """
    
    # Verifying if the list of tickers is compatible
    available = client.get_exchange_symbols("US")

    available = set(i['Code'] for i in available)
    
    tickers = [i for i in tickers if i in available]
    
    # Iterating through the tickers
    dfs = {}
    
    for ticker in tickers:
        
        dfs[ticker] = formatFundamentals(ticker)
        
    
    return pd.concat(dfs, axis=0)
    

In [358]:
df = getMultipleFunds(["AAPL", "TSLA", "MSFT"], api_key)

In [364]:
df.loc['TSLA']

,currency_symbol,totalAssets,intangibleAssets,earningAssets,otherCurrentAssets,totalLiab,totalStockholderEquity,deferredLongTermLiab,otherCurrentLiab,commonStock,...,preferredStockAndOtherAdjustments,beforeAfterMarket,currency,epsActual,epsEstimate,epsDifference,surprisePercent,adjusted_close,close,volume
2008-09-30,None,51699000.0,None,None,None,251413000.0,None,None,None,None,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-12-31,USD,51699000.0,None,None,2180000.0,251413000.0,None,None,71607000.0,7000.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-03-31,None,None,None,None,None,None,None,None,None,None,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-06-30,USD,155916000.0,None,None,None,386561000.0,None,None,None,None,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-09-30,USD,155916000.0,None,None,None,386561000.0,None,None,None,None,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12-31,USD,130424000.0,None,None,4222000.0,383947000.0,65702000.0,None,41661000.0,7000.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31,USD,145320000.0,None,None,4537000.0,101141000.0,39928000.0,None,40143000.0,8000.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-30,USD,147974000.0,None,None,6745000.0,457749000.0,9450000.0,None,41123000.0,8000.0,...,None,None,USD,-0.0812,None,None,None,4.7660,23.830,85971954.0
2010-09-30,USD,361621000.0,None,None,8870000.0,141698000.0,219923000.0,None,39727000.0,93000.0,...,None,None,USD,-0.3700,-0.4300,0.0600,13.9535,4.0810,20.405,10983125.0
2010-12-31,USD,386082000.0,14508000.0,None,10839000.0,179034000.0,207048000.0,None,53661000.0,95000.0,...,None,None,USD,-0.4700,-0.5000,0.0300,6,5.3260,26.630,7089235.0
